In [1]:
import tensorflow as tf
import keras

from keras import models, layers
from keras.models import Model, model_from_json, Sequential

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, SeparableConv2D, UpSampling2D, BatchNormalization, Input, GlobalAveragePooling2D

In [2]:
def entry_flow(inputs) :

    x = Conv2D(32, 3, strides = 2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(64,3,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    previous_block_activation = x

    for size in [128, 256, 728] :

        x = Activation('relu')(x)
        x = SeparableConv2D(size, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(size, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding='same')(x)

        residual = Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)

        x = keras.layers.Add()([x, residual])
        previous_block_activation = x

    return x

In [3]:
def middle_flow(x, num_blocks=8) :

    previous_block_activation = x

    for _ in range(num_blocks) :

        x = Activation('relu')(x)
        x = SeparableConv2D(728, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(728, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(728, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = keras.layers.Add()([x, previous_block_activation])
        previous_block_activation = x

    return x

In [4]:
def exit_flow(x) :

    previous_block_activation = x

    x = Activation('relu')(x)
    x = SeparableConv2D(728, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = Activation('relu')(x)
    x = SeparableConv2D(1024, 3, padding='same')(x) 
    x = BatchNormalization()(x)

    x = MaxPooling2D(3, strides=2, padding='same')(x)

    residual = Conv2D(1024, 1, strides=2, padding='same')(previous_block_activation)
    x = keras.layers.Add()([x, residual])

    x = Activation('relu')(x)
    x = SeparableConv2D(728, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = Activation('relu')(x)
    x = SeparableConv2D(1024, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='sigmoid')(x)

    return x

In [5]:
inputs = Input(shape=(299, 299, 3))
outputs = exit_flow(middle_flow(entry_flow(inputs)))
model = Model(inputs, outputs)

In [6]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 150, 150, 32)         896       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 150, 150, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 150, 150, 32)         0         ['batch_normalization[0][0

                                                                                                  
 max_pooling2d_2 (MaxPoolin  (None, 19, 19, 728)          0         ['batch_normalization_7[0][0]'
 g2D)                                                               ]                             
                                                                                                  
 conv2d_4 (Conv2D)           (None, 19, 19, 728)          187096    ['add_1[0][0]']               
                                                                                                  
 add_2 (Add)                 (None, 19, 19, 728)          0         ['max_pooling2d_2[0][0]',     
                                                                     'conv2d_4[0][0]']            
                                                                                                  
 activation_8 (Activation)   (None, 19, 19, 728)          0         ['add_2[0][0]']               
          

In [7]:
num_classes = 2  


# training parameters
EPOCHS = 100
BATCH_SIZE = 32
image_height = 299
image_width = 299

# Шляхи до директорій для тренування та валідації для кожного класу
train_dir = "./brandDataset/train"
valid_dir = "./brandDataset/validation"

# Зазначення назв класів
class_names = ['apple', 'others']


train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=10,
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  shear_range=0.1,
                  zoom_range=0.1)

# Зміна параметрів генераторів для відповідних кількостей класів
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="binary",  
                                                    classes=class_names)

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode="binary",  
                                                    classes=class_names)

train_num = train_generator.samples
valid_num = valid_generator.samples

Found 632 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


In [8]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    steps_per_epoch=train_num // BATCH_SIZE,
                    validation_data=valid_generator,
                    validation_steps=valid_num // BATCH_SIZE,
                    verbose=1)

Epoch 1/100


 1/19 [>.............................] - ETA: 11:27 - loss: 0.6818 - accuracy: 0.5938

c:\Labs\NeuralNetworkLabs\.venv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


19/19 [==============================] - 265s 13s/step - loss: 0.8791 - accuracy: 0.5417 - val_loss: 0.7167 - val_accuracy: 0.5000
Epoch 2/100
19/19 [==============================] - 239s 13s/step - loss: 0.6916 - accuracy: 0.5983 - val_loss: 0.7274 - val_accuracy: 0.5104
Epoch 3/100
19/19 [==============================] - 234s 12s/step - loss: 0.7076 - accuracy: 0.6450 - val_loss: 0.7261 - val_accuracy: 0.5208
Epoch 4/100
19/19 [==============================] - 240s 13s/step - loss: 0.6300 - accuracy: 0.6467 - val_loss: 0.7307 - val_accuracy: 0.5104
Epoch 5/100
19/19 [==============================] - 235s 12s/step - loss: 0.6407 - accuracy: 0.6500 - val_loss: 0.7581 - val_accuracy: 0.4479
Epoch 6/100
19/19 [==============================] - 237s 12s/step - loss: 0.6087 - accuracy: 0.6583 - val_loss: 0.7361 - val_accuracy: 0.5000
Epoch 7/100
19/19 [==============================] - 236s 12s/step - loss: 0.6417 - accuracy: 0.6217 - val_loss: 0.7392 - val_accuracy: 0.4896
Epoch 8/100

In [9]:
# Збереження моделі
model.save('logo_detection_model.h5')

c:\Labs\NeuralNetworkLabs\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
import cv2
import numpy as np
from keras.models import load_model

# Завантаження навченої моделі
model = load_model('logo_detection_model.h5')

# Відкриття відеофайлу
video_capture = cv2.VideoCapture('my_video.mp4')
img_size = (299, 299)

# Визначення властивостей відео
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

# Цикл обробки кадрів
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Переведення зображення до формату, який можна використовувати для моделі
    input_image = cv2.resize(frame, img_size)
    input_image = np.expand_dims(input_image, axis=0) / 255.0

    # Передбачення за допомогою моделі
    predictions = model.predict(input_image)
    print("Predictions:", predictions)

    # Якщо логотип визначено на кадрі, вивести час
    if predictions[0, class_names.index('apple')] > 0.5297:
        current_frame = int(video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        current_time = current_frame / fps
        print(f"Logo detected at time: {current_time} seconds")

    # Відображення відео
    cv2.imshow('Video', frame)

    # Вихід з циклу при натисканні клавіші 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Закриття відеоструму та вікна
video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 1s/step
Predictions: [[0.49526945]]
1/1 [==============================] - 0s 184ms/step
Predictions: [[0.5044345]]
1/1 [==============================] - 0s 176ms/step
Predictions: [[0.5041316]]
1/1 [==============================] - 0s 164ms/step
Predictions: [[0.5040687]]
1/1 [==============================] - 0s 163ms/step
Predictions: [[0.50455034]]
1/1 [==============================] - 0s 165ms/step
Predictions: [[0.5046581]]
1/1 [==============================] - 0s 165ms/step
Predictions: [[0.50456876]]
1/1 [==============================] - 0s 175ms/step
Predictions: [[0.50516516]]
1/1 [==============================] - 0s 179ms/step
Predictions: [[0.50524485]]
1/1 [==============================] - 0s 161ms/step
Predictions: [[0.5095565]]
1/1 [==============================] - 0s 170ms/step
Predictions: [[0.50699633]]
1/1 [==============================] - 0s 171ms/step
Predictions: [[0.5107432]]
1/1 [==============================] 